https://pyquantnews.podia.com/community/topics/63713/posts/233545-cboe-options-data-update-thanks-to-the-community-feedback-spx-load-processing-time-is-now-about-4

In [1]:
"""CBOE Model Functions"""
__docformat__ = "numpy"

In [2]:
import logging
import pandas as pd
import requests
from typing import Tuple
from datetime import datetime
from requests.exceptions import HTTPError
from openbb_terminal.helper_funcs import request

In [3]:
from openbb_terminal.rich_config import console

pd.set_option("display.max_columns", None)
pd.set_option("display.max_colwidth", None)
pd.set_option("display.max_rows", None)

In [4]:
logger = logging.getLogger(__name__)

In [5]:
TICKER_EXCEPTIONS = ["NDX", "RUT"]

List of US Indexes from the CBOE - DataFrame is passed downstream automatically as "indexes" for exception handling

In [6]:
def get_cboe_indexes() -> pd.DataFrame:
    """Gets the CBOE Indexes for exception handling"""

    indices = pd.read_json(
        "https://cdn.cboe.com/api/global/us_indices/definitions/all_indices.json"
    )
    indices = pd.DataFrame(indices).rename(
        columns={
            "calc_end_time": "Close Time",
            "calc_start_time": "Open Time",
            "currency": "Currency",
            "description": "Description",
            "display": "Display",
            "featured": "Featured",
            "featured_order": "Featured Order",
            "index_symbol": "Ticker",
            "mkt_data_delay": "Data Delay",
            "name": "Name",
            "tick_days": "Tick Days",
            "tick_frequency": "Frequency",
            "tick_period": "Period",
            "time_zone": "Time Zone",
        },
    )

    indices_order = [
        "Ticker",
        "Description",
        "Currency",
        "Tick Days",
        "Frequency",
        "Period",
        "Time Zone",
    ]
    indices = indices.set_index(keys=["Name"])

    cboe_us_indices = pd.DataFrame(indices, columns=indices_order)
    indexes = list(cboe_us_indices.Ticker[:])

    return indexes

In [7]:
indexes = get_cboe_indexes()

In [8]:
def get_top_of_book(
    symbol: str, exchange: str = "BZX"
) -> Tuple[pd.DataFrame, pd.DataFrame]:
    """Get top of book bid and ask for ticker on exchange [CBOE.com]

    Parameters
    ----------
    symbol: str
        Ticker to get
    exchange: str
        Exchange to look at.  Can be `BZX`,`EDGX`, `BYX`, `EDGA`

    Returns
    -------
    pd.DatatFrame
        Dataframe of Bids
    pd.DataFrame
        Dataframe of asks

    """
    if exchange not in ["BZX", "EDGX", "BYX", "EDGA"]:
        console.print(f"[red]Exchange not valid: {exchange}[/red]")
        return pd.DataFrame(), pd.DataFrame()
    # exchange need to be lower case.  Not sure why
    url = f"https://www.cboe.com/json/{exchange.lower()}/book/{symbol}"

    r = request(
        url,
        headers={
            "User-Agent": "Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko)"
            " Chrome/106.0.0.0 Safari/537.36",
            "referer": "https://www.cboe.com/us/equities/market_statistics/book_viewer/",
        },
    )
    if r.status_code != 200:
        console.print(f"[red]Request failed with code {r.status_code}[/red]")
        return pd.DataFrame(), pd.DataFrame()
    r_json = r.json()
    if r_json["data"]["company"] == "unknown symbol":
        console.print(f"[red]Unknown symbol: {symbol}[/red]")
        return pd.DataFrame(), pd.DataFrame()
    bids = pd.DataFrame(r_json["data"]["bids"], columns=["Size", "Price"])
    asks = pd.DataFrame(r_json["data"]["asks"], columns=["Size", "Price"])
    if bids.empty or asks.empty:
        console.print(
            "[red]No bid/ask data. Note this is real time so there is no data after market close.[/red]"
        )
    return bids, asks

    # Get Ticker Info and Expirations

In [9]:
def get_ticker_info(symbol: str) -> Tuple[pd.DataFrame, pd.Series]:
    """Gets basic info for the symbol and expiration dates

    Parameters
    ----------
    symbol: str
        The ticker to lookup

    Returns
    -------
    Tuple[pd.DataFrame, pd.Series]: ticker_details,ticker_expirations

    Examples
    --------
    ticker_details,ticker_expirations = get_ticker_info('AAPL')

    ticker_details,ticker_expirations = get_ticker_info('VIX')
    """

    # Variables for exception handling

    stock = "stock"
    index = "index"
    ticker = symbol

    try:
        # Checks ticker to determine if ticker is an index or an exception that requires modifying the request's URLs

        if ticker in TICKER_EXCEPTIONS:
            new_ticker = "^" + ticker
        else:
            if ticker not in indexes:
                new_ticker = ticker

            elif ticker in indexes:
                new_ticker = "^" + ticker

                # Gets the data to return, and if none returns empty Tuple #

        symbol_info_url = (
            "https://www.cboe.com/education/tools/trade-optimizer/symbol-info/?symbol="
            f"{new_ticker}"
        )

        symbol_info = requests.get(symbol_info_url)
        symbol_info_json = pd.Series(symbol_info.json())

        if symbol_info_json.success is False:
            ticker_details = pd.DataFrame()
            ticker_expirations = []
            console.print("[red]No data found for the symbol: " f"{ticker}" "[/red]")
        else:
            symbol_details = pd.Series(symbol_info_json["details"])
            symbol_details = pd.DataFrame(symbol_details).transpose()
            symbol_details = symbol_details.reset_index()
            ticker_expirations = pd.Series(symbol_info_json["expirations"])

            # Cleans columns depending on if the security type is a stock or an index

            type: str = symbol_details.security_type

            if stock[0] in type[0]:
                stock_details = symbol_details
                ticker_details = pd.DataFrame(stock_details).rename(
                    columns={
                        "symbol": "Symbol",
                        "current_price": "Current Price",
                        "bid": "Bid",
                        "ask": "Ask",
                        "bid_size": "Bid Size",
                        "ask_size": "Ask Size",
                        "open": "Open",
                        "high": "High",
                        "low": "Low",
                        "close": "Close",
                        "volume": "Volume",
                        "iv30": "IV30",
                        "prev_day_close": "Previous Close",
                        "price_change": "Price Change",
                        "price_change_percent": "Price Change %",
                        "iv30_change": "IV30 Change",
                        "iv30_percent_change": "IV30 Change %",
                        "last_trade_time": "Last Trade Time",
                        "exchange_id": "Exchange ID",
                        "tick": "Tick",
                        "security_type": "Type",
                    }
                )
                details_columns = [
                    "Symbol",
                    "Type",
                    "Tick",
                    "Bid",
                    "Bid Size",
                    "Ask Size",
                    "Ask",
                    "Current Price",
                    "Open",
                    "High",
                    "Low",
                    "Close",
                    "Volume",
                    "Previous Close",
                    "Price Change",
                    "Price Change %",
                    "IV30",
                    "IV30 Change",
                    "IV30 Change %",
                    "Last Trade Time",
                ]
                ticker_details = (
                    pd.DataFrame(ticker_details, columns=details_columns)
                    .set_index(keys="Symbol")
                    .dropna(axis=1)
                    .transpose()
                )

            if index[0] in type[0]:
                index_details = symbol_details
                ticker_details = pd.DataFrame(index_details).rename(
                    columns={
                        "symbol": "Symbol",
                        "security_type": "Type",
                        "current_price": "Current Price",
                        "price_change": "Change",
                        "price_change_percent": "Change %",
                        "tick": "Tick",
                        "open": "Open",
                        "high": "High",
                        "low": "Low",
                        "close": "Close",
                        "prev_day_close": "Previous Close",
                        "iv30": "IV30",
                        "iv30_change": "IV30 Change",
                        "iv30_change_percent": "IV30 Change %",
                        "last_trade_time": "Last Trade Time",
                    }
                )

                index_columns = [
                    "Symbol",
                    "Type",
                    "Tick",
                    "Current Price",
                    "Open",
                    "High",
                    "Low",
                    "Close",
                    "Previous Close",
                    "Change",
                    "Change %",
                    "IV30",
                    "IV30 Change",
                    "IV30 Change %",
                    "Last Trade Time",
                ]

                ticker_details = (
                    pd.DataFrame(ticker_details, columns=index_columns)
                    .set_index(keys="Symbol")
                    .dropna(axis=1)
                    .transpose()
                )

    except HTTPError:
        console.print("[red]There was an error with the request[/red]'\n")
        ticker_details = pd.DataFrame()
        ticker_expirations: object = pd.Series()

    return ticker_details, ticker_expirations

    # Gets annualized high/low historical and implied volatility over 30/60/90 day windows.

In [10]:
def get_ticker_iv(symbol: str) -> pd.DataFrame:
    """Gets annualized high/low historical and implied volatility over 30/60/90 day windows.

    Parameters
    ----------
    symbol: str
        The loaded ticker

    Returns
    -------
    pd.DataFrame: ticker_iv

    Examples
    --------
    ticker_iv = get_ticker_iv('AAPL')

    ticker_iv = get_ticker_iv('NDX')
    """

    ticker = symbol

    # Checks ticker to determine if ticker is an index or an exception that requires modifying the request's URLs
    try:
        if ticker in TICKER_EXCEPTIONS:
            quotes_iv_url = (
                "https://cdn.cboe.com/api/global/delayed_quotes/historical_data/_"
                f"{ticker}"
                ".json"
            )
        else:
            if ticker not in indexes:
                quotes_iv_url = (
                    "https://cdn.cboe.com/api/global/delayed_quotes/historical_data/"
                    f"{ticker}"
                    ".json"
                )

            elif ticker in indexes:
                quotes_iv_url = (
                    "https://cdn.cboe.com/api/global/delayed_quotes/historical_data/_"
                    f"{ticker}"
                    ".json"
                )

                # Gets annualized high/low historical and implied volatility over 30/60/90 day windows.

        h_iv = requests.get(quotes_iv_url)

        if h_iv.status_code != 200:
            console.print("[red]No data found for the symbol: " f"{ticker}" "[/red]")
            return pd.DataFrame()

        else:
            h_iv_json = pd.DataFrame(h_iv.json())
            h_columns = [
                "annual_high",
                "annual_low",
                "hv30_annual_high",
                "hv30_annual_low",
                "hv60_annual_high",
                "hv60_annual_low",
                "hv90_annual_high",
                "hv90_annual_low",
                "iv30_annual_high",
                "iv30_annual_low",
                "iv60_annual_high",
                "iv60_annual_low",
                "iv90_annual_high",
                "iv90_annual_low",
                "symbol",
            ]
            h_data = h_iv_json[1:]
            h_data = pd.DataFrame(h_iv_json).transpose()
            h_data = h_data[1:2]
            quotes_iv_df = pd.DataFrame(data=h_data, columns=h_columns).reset_index()

            quotes_iv_df = pd.DataFrame(quotes_iv_df).rename(
                columns={
                    "annual_high": "1Y High",
                    "annual_low": "1Y Low",
                    "hv30_annual_high": "HV30 1Y High",
                    "hv30_annual_low": "HV30 1Y Low",
                    "hv60_annual_high": "HV60 1Y High",
                    "hv60_annual_low": "HV60 1Y Low",
                    "hv90_annual_high": "HV90 1Y High",
                    "hv90_annual_low": "HV90 1Y Low",
                    "iv30_annual_high": "IV30 1Y High",
                    "iv30_annual_low": "IV30 1Y Low",
                    "iv60_annual_high": "IV60 1Y High",
                    "iv60_annual_low": "IV60 1Y Low",
                    "iv90_annual_high": "IV90 1Y High",
                    "iv90_annual_low": "IV90 1Y Low",
                    "symbol": "Symbol",
                },
            )

            quotes_iv_df = quotes_iv_df.set_index(keys="Symbol")

            iv_order = [
                "IV30 1Y High",
                "HV30 1Y High",
                "IV30 1Y Low",
                "HV30 1Y Low",
                "IV60 1Y High",
                "HV60 1Y High",
                "IV60 1Y Low",
                "HV60 1Y low",
                "IV90 1Y High",
                "HV90 1Y High",
                "IV90 1Y Low",
                "HV 90 1Y Low",
            ]

            ticker_iv = (
                pd.DataFrame(quotes_iv_df, columns=iv_order)
                .fillna(value="N/A")
                .transpose()
            )
    except HTTPError:
        console.print("[red]There was an error with the request[/red]'\n")

    return ticker_iv

    # Gets quotes and greeks data and returns a dataframe: options_quotes

In [11]:
def get_ticker_chains(symbol: str) -> pd.DataFrame:
    """Gets the complete options chains for a ticker

    Parameters
    ----------
    symbol: str
        The ticker get options data for

    Returns
    -------
    ticker_options: pd.DataFrame
        DataFrame of all options chains for the ticker

    Examples
    --------
    ticker_options = get_ticker_chains('SPX')

    ticker_options = get_ticker_chains('SPX').filter(like = '2027-12-17', axis = 0)

    ticker_calls = get_ticker_chains('AAPL').filter(like = 'Call', axis = 0)

    vix_20C = (
        get_ticker_chains('VIX')
        .filter(like = 'Call', axis = 0)
        .reset_index(['Expiration', 'Type'])
        .query('20.0')
    )
    """

    ticker = symbol

    # Checks ticker to determine if ticker is an index or an exception that requires modifying the request's URLs

    try:

        ticker_info, _ = get_ticker_info(ticker)
        if not ticker_info.empty:
            last_price = float(ticker_info.loc["Current Price"])
        else:
            return pd.DataFrame()

        if ticker in TICKER_EXCEPTIONS:
            quotes_url = (
                "https://cdn.cboe.com/api/global/delayed_quotes/options/_"
                f"{ticker}"
                ".json"
            )
        else:
            if ticker not in indexes:
                quotes_url = (
                    "https://cdn.cboe.com/api/global/delayed_quotes/options/"
                    f"{ticker}"
                    ".json"
                )
            if ticker in indexes:
                quotes_url = (
                    "https://cdn.cboe.com/api/global/delayed_quotes/options/_"
                    f"{ticker}"
                    ".json"
                )

        r = requests.get(quotes_url)
        if r.status_code != 200:
            console.print("[red]No data found for the symbol: " f"{ticker}" "[/red]")
            return pd.DataFrame()
        else:
            r_json = r.json()
            data = pd.DataFrame(r_json["data"])
            options = pd.Series(data.options, index=data.index)
            options_columns = list(options[0])
            options_data = list(options[:])
            options_df = pd.DataFrame(options_data, columns=options_columns)
            options_df = pd.DataFrame(options_df).rename(
                columns={
                    "option": "Option Symbol",
                    "bid": "Bid",
                    "bid_size": "Bid Size",
                    "ask": "Ask",
                    "ask_size": "Ask Size",
                    "iv": "IV",
                    "open_interest": "OI",
                    "volume": "Vol",
                    "delta": "Delta",
                    "gamma": "Gamma",
                    "theta": "Theta",
                    "rho": "Rho",
                    "vega": "Vega",
                    "theo": "Theoretical",
                    "change": "Change",
                    "open": "Open",
                    "high": "High",
                    "low": "Low",
                    "tick": "Tick",
                    "last_trade_price": "Last Price",
                    "last_trade_time": "Timestamp",
                    "percent_change": "% Change",
                    "prev_day_close": "Prev Close",
                }
            )

            options_df_order = [
                "Option Symbol",
                "Tick",
                "Theoretical",
                "Last Price",
                "Prev Close",
                "% Change",
                "Open",
                "High",
                "Low",
                "Bid Size",
                "Bid",
                "Ask",
                "Ask Size",
                "Vol",
                "OI",
                "IV",
                "Theta",
                "Delta",
                "Gamma",
                "Vega",
                "Rho",
                "Timestamp",
            ]

            options_df = pd.DataFrame(options_df, columns=options_df_order).set_index(
                keys=["Option Symbol"]
            )

            option_df_index = pd.Series(options_df.index).str.extractall(
                r"^(?P<Ticker>\D*)(?P<Expiration>\d*)(?P<Type>\D*)(?P<Strike>\d*)"
            )

            option_df_index = option_df_index.reset_index().drop(
                columns=["match", "level_0"]
            )

            option_df_index.Expiration = pd.DatetimeIndex(
                option_df_index.Expiration, yearfirst=True
            )

            option_df_index.Type = option_df_index.Type.str.replace(
                "C", "Call"
            ).str.replace("P", "Put")

            option_df_index.Strike = [ele.lstrip("0") for ele in option_df_index.Strike]
            option_df_index.Strike = option_df_index.Strike.astype(float)
            option_df_index.Strike = option_df_index.Strike * (1 / 1000)
            option_df_index = option_df_index.drop(columns=["Ticker"])
            ticker_chains = option_df_index.join(options_df.reset_index())

            ticker_chains = ticker_chains.drop(columns=["Option Symbol"]).set_index(
                keys=["Expiration", "Strike", "Type"]
            )

            ticker_chains["Theoretical"] = round(
                ticker_chains["Theoretical"], ndigits=2
            )
            ticker_chains["Prev Close"] = round(ticker_chains["Prev Close"], ndigits=2)
            ticker_chains["% Change"] = round(ticker_chains["% Change"], ndigits=4)

            ticker_chains.Tick = ticker_chains["Tick"].str.replace(
                pat="no_change", repl="No Change"
            )

            ticker_chains.OI = ticker_chains["OI"].astype(int)
            ticker_chains.Vol = ticker_chains["Vol"].astype(int)
            ticker_chains["Bid Size"] = ticker_chains["Bid Size"].astype(int)
            ticker_chains["Ask Size"] = ticker_chains["Ask Size"].astype(int)
            ticker_chains = ticker_chains.sort_index()
            ticker_calls = ticker_chains.filter(like="Call", axis=0).copy()
            ticker_puts = ticker_chains.filter(like="Put", axis=0).copy()
            ticker_calls = ticker_calls.reset_index()

            ticker_calls.loc[:, ("$ to Spot")] = round(
                (ticker_calls.loc[:, ("Strike")])
                + (ticker_calls.loc[:, ("Ask")])
                - (last_price),
                ndigits=2,
            )

            ticker_calls.loc[:, ("% to Spot")] = round(
                (ticker_calls.loc[:, ("$ to Spot")] / last_price) * 100, ndigits=4
            )

            ticker_calls.loc[:, ("Breakeven")] = (
                ticker_calls.loc[:, ("Strike")] + ticker_calls.loc[:, ("Ask")]
            )

            ticker_calls.loc[:, ("Delta $")] = (
                (ticker_calls.loc[:, ("Delta")] * 100)
                * (ticker_calls.loc[:, ("OI")])
                * last_price
            )

            ticker_calls.loc[:, ("GEX")] = (
                ticker_calls.loc[:, ("Gamma")]
                * 100
                * ticker_calls.loc[:, ("OI")]
                * (last_price * last_price)
                * 0.01
            )

            ticker_calls.GEX = ticker_calls.GEX.astype(int)
            ticker_calls["Delta $"] = ticker_calls["Delta $"].astype(int)
            ticker_calls = ticker_calls.set_index(keys=["Expiration", "Strike", "Type"])

            ticker_puts = ticker_puts.reset_index()

            ticker_puts.loc[:, ("$ to Spot")] = round(
                (ticker_puts.loc[:, ("Strike")])
                - (ticker_puts.loc[:, ("Ask")])
                - (last_price),
                ndigits=2,
            )

            ticker_puts.loc[:, ("% to Spot")] = round(
                (ticker_puts.loc[:, ("$ to Spot")] / last_price) * 100, ndigits=4
            )

            ticker_puts.loc[:, ("Breakeven")] = (
                ticker_puts.loc[:, ("Strike")] - ticker_puts.loc[:, ("Ask")]
            )

            ticker_puts.loc[:, ("Delta $")] = (
                (ticker_puts.loc[:, ("Delta")] * 100)
                * (ticker_puts.loc[:, ("OI")])
                * last_price
                * (-1)
            )

            ticker_puts.loc[:, ("GEX")] = (
                ticker_puts.loc[:, ("Gamma")]
                * 100
                * ticker_puts.loc[:, ("OI")]
                * (last_price * last_price)
                * 0.01
            )

            ticker_puts.GEX = ticker_puts.GEX.astype(int)
            ticker_puts["Delta $"] = ticker_puts["Delta $"].astype(int)
            ticker_puts.set_index(keys=["Expiration", "Strike", "Type"], inplace=True)

            ticker_chains = pd.concat([ticker_puts, ticker_calls]).sort_index()

            temp = ticker_chains.reset_index().get(["Expiration"])
            temp.Expiration = pd.DatetimeIndex(data=temp.Expiration)
            temp_ = temp.Expiration - datetime.now()
            temp_ = temp_.astype(str)
            temp_ = temp_.str.extractall(r"^(?P<DTE>\d*)")
            temp_ = temp_.droplevel("match")
            temp_.DTE = temp_.DTE.fillna("-1")
            temp_.DTE = temp_.DTE.astype(int)
            temp_.DTE = temp_.DTE + 1
            ticker_chains = temp_.join(ticker_chains.reset_index()).set_index(
                ["Expiration", "Strike", "Type"]
            )

            ticker_chains_cols = [
                "DTE",
                "Tick",
                "Last Price",
                "% Change",
                "Theoretical",
                "$ to Spot",
                "% to Spot",
                "Breakeven",
                "Vol",
                "OI",
                "Delta $",
                "GEX",
                "IV",
                "Theta",
                "Delta",
                "Gamma",
                "Vega",
                "Rho",
                "Open",
                "High",
                "Low",
                "Prev Close",
                "Bid Size",
                "Bid",
                "Ask",
                "Ask Size",
                "Timestamp",
            ]

            ticker_chains = pd.DataFrame(data=ticker_chains, columns=ticker_chains_cols)

    except HTTPError:
        console.print("[red]There was an error with the request[/red]'\n")

    return ticker_chains

In [12]:
def separate_chains(chains_df: pd.DataFrame) -> Tuple[pd.DataFrame, pd.DataFrame]:
    """Helper function to separate Options Chains into Call and Put Chains.
    Parameters
    ----------
    chains_df : pd.DataFrame
        DataFrame of options chains data.

    Returns
    -------
    Tuple: [pd.DataFrame, pd.DataFrame]
        Tuple of options DataFrames separated by calls and puts.

    Example
    -------
    calls,puts = separate_chains(chains_df)
    """

    if not chains_df.empty and not chains_df is None:
        calls: pd.DataFrame = chains_df.filter(like="Call", axis=0).copy()
        puts: pd.DataFrame = chains_df.filter(like="Put", axis=0).copy()

        return calls, puts

    else:
        console.print(
            "[red]There was an error with the input data, or the DataFrame passed was empty.[/red]"
            "\n"
        )
        calls = pd.DataFrame()
        puts = pd.DataFrame()
        return calls, puts

In [13]:
def calc_chains_by_expiration(chains_df: pd.DataFrame) -> pd.DataFrame:
    """Calculates stats for options chains by expiration.
    Parameters
    ----------
    chains_df: pd.DataFrame
        DataFrame of options chains to use.

    Returns
    -------
    pd.DataFrame
        DataFrame with stats by expiration date.

    Example
    -------
    chains_by_expiration = calc_chains_by_expiration(chains_df)
    """

    if not chains_df.empty and not chains_df is None:

        calls, puts = separate_chains(chains_df)

        calls_by_expiration = (
            calls.reset_index()
            .groupby('Expiration')
            .sum(numeric_only = True)[['OI', 'Vol', 'Delta $', 'GEX']]
        )

        calls_by_expiration = calls_by_expiration.rename(
            columns = {
                'OI': 'Call OI',
                'Vol': 'Call Vol',
                'Delta $': 'Call Delta $',
                'GEX': 'Call GEX',
            }
        )

        puts_by_expiration = (
            puts.reset_index()
            .groupby('Expiration')
            .sum(numeric_only = True)[['OI', 'Vol', 'Delta $', 'GEX']]
        )

        puts_by_expiration['Delta $'] = puts_by_expiration['Delta $'] * (-1)
        puts_by_expiration['GEX'] = puts_by_expiration['GEX'] * (-1)

        puts_by_expiration = puts_by_expiration.rename(
            columns = {
                'OI': 'Put OI',
                'Vol': 'Put Vol',
                'Delta $': 'Put Delta $',
                'GEX': 'Put GEX',
            }
        )
        
        chains_by_expiration = calls_by_expiration.join(puts_by_expiration)

        chains_by_expiration["OI Ratio"] = round(
            chains_by_expiration["Put OI"] / chains_by_expiration["Call OI"], ndigits=4
        )

        chains_by_expiration["Net OI"] = (
            chains_by_expiration["Call OI"] + chains_by_expiration["Put OI"]
        )

        chains_by_expiration["Vol Ratio"] = round(
            chains_by_expiration["Put Vol"] / chains_by_expiration["Call Vol"],
            ndigits=4,
        )

        chains_by_expiration["Net Vol"] = (
            chains_by_expiration["Call Vol"] + chains_by_expiration["Put Vol"]
        )

        chains_by_expiration["Vol-OI Ratio"] = round(
            chains_by_expiration["Net Vol"] / chains_by_expiration["Net OI"], ndigits=4
        )

        chains_by_expiration["Net Delta $"] = (
            chains_by_expiration["Call Delta $"] + chains_by_expiration["Put Delta $"]
        )

        chains_by_expiration["Net GEX"] = (
            chains_by_expiration["Call GEX"] + chains_by_expiration["Put GEX"]
        )

        cols_order = [
            "Call OI",
            "Put OI",
            "Net OI",
            "OI Ratio",
            "Call Vol",
            "Put Vol",
            "Net Vol",
            "Vol Ratio",
            "Vol-OI Ratio",
            "Call Delta $",
            "Put Delta $",
            "Net Delta $",
            "Call GEX",
            "Put GEX",
            "Net GEX",
        ]

        chains_by_expiration = (
            pd.DataFrame(chains_by_expiration, columns = cols_order)
        )
        
        return chains_by_expiration

    else:
        console.print(
            "[red]There was an error with the input data, or the DataFrame passed was empty.[/red]"
            "\n"
        )
        chains_by_expiration = pd.DataFrame()
        return chains_by_expiration

In [14]:
def calc_chains_by_strike(chains_df: pd.DataFrame) -> pd.DataFrame:
    """
    Parameters
    ----------
    chains_df: pd.DataFrame
        Dataframe of the chains by expiration

    Returns
    -------
    pd.DataFrame:
        Dataframe of the chains by strike        
    
    Example
    -------
    chains_by_strike = calc_chains_by_strike(chains_df)
    """
    
    if not chains_df.empty and not chains_df is None:

        calls, puts = separate_chains(chains_df)

        calls_by_strike = (
            calls.reset_index()
            .groupby('Strike')
            .sum(numeric_only = True)[['OI', 'Vol', 'Delta $', 'GEX']]
        )
        
        calls_by_strike = calls_by_strike.rename(
            columns = {
                'OI': 'Call OI',
                'Vol': 'Call Vol',
                'Delta $': 'Call Delta $',
                'GEX': 'Call GEX',
            }
        )

        puts_by_strike = (
            puts.reset_index()
            .groupby('Strike')
            .sum(numeric_only = True)[['OI', 'Vol', 'Delta $', 'GEX']]
        )
        
        puts_by_strike['Delta $'] = puts_by_strike['Delta $'] * (-1)
        puts_by_strike['GEX'] = puts_by_strike['GEX'] * (-1)
        
        puts_by_strike = puts_by_strike.rename(
            columns = {
                'OI': 'Put OI',
                'Vol': 'Put Vol',
                'Delta $': 'Put Delta $',
                'GEX': 'Put GEX',
            }
        )
        chains_by_strike = calls_by_strike.join(puts_by_strike)

        chains_by_strike["Net OI"] = (
            chains_by_strike["Call OI"] + chains_by_strike["Put OI"]
        )
        chains_by_strike["Net Vol"] = (
            chains_by_strike["Call Vol"] + chains_by_strike["Put Vol"]
        )
        chains_by_strike["Net Delta $"] = (
            chains_by_strike["Call Delta $"] + chains_by_strike["Put Delta $"]
        )
        chains_by_strike["Net GEX"] = (
            chains_by_strike["Call GEX"] + chains_by_strike["Put GEX"]
        )

        cols_order = [
            "Call OI",
            "Put OI",
            "Net OI",
            "Call Vol",
            "Put Vol",
            "Net Vol",
            "Call Delta $",
            "Put Delta $",
            "Net Delta $",
            "Call GEX",
            "Put GEX",
            "Net GEX",
        ]

        chains_by_strike = pd.DataFrame(data=chains_by_strike, columns=cols_order)

        return chains_by_strike

    else:
        console.print(
            "[red]There was an error with the input data, or the DataFrame passed was empty.[/red]"
            "\n"
        )
        chains_by_strike = pd.DataFrame()
        return chains_by_strike

In [22]:
ticker = 'SPY'

In [23]:
chains = get_ticker_chains(ticker)

chains

DTE       Tick  Last Price  % Change  Theoretical  \
Expiration Strike Type                                                       
2023-02-01 320.0  Call     0         up       93.33   10.0719        91.52   
                  Put      0  No Change        0.01  100.0000         0.00   
           330.0  Call     0         up       82.97   10.9225        81.52   
                  Put      0       down        0.02    0.0000         0.00   
           331.0  Call     0         up       82.33   11.5583        80.52   
...                      ...        ...         ...       ...          ...   
2025-12-19 640.0  Put   1052  No Change        0.00    0.0000       228.48   
           645.0  Call  1052         up        3.10    3.8526         3.71   
                  Put   1052  No Change        0.00    0.0000       233.48   
           650.0  Call  1052       down        3.60   18.0328         3.51   
                  Put   1052       down      235.98   -3.6816       238.48   

                        $ to Spot  % to Spot  Breakeven  Vol   OI  ...  \
Expiration Strike Type                                             ...   
2023-02-01 320.0  Call       1.24     0.3019     412.04    6    0  ...   
                  Put      -90.81   -22.1056     319.99    2  293  ...   
           330.0  Call       1.24     0.3019     412.04    5    0  ...   
                  Put      -80.81   -19.6714     329.99    0  355  ...   
           331.0  Call       1.24     0.3019     412.04    4    0  ...   
...                           ...        ...        ...  ...  ...  ...   
2025-12-19 640.0  Put       -4.46    -1.0857     406.34    0    0  ...   
           645.0  Call     243.80    59.3476     654.60    1   15  ...   
                  Put       -4.35    -1.0589     406.45    0    0  ...   
           650.0  Call     243.09    59.1748     653.89   22  207  ...   
                  Put       -3.95    -0.9615     406.85    1    1  ...   

                           Rho    Open    High     Low  Prev Close  Bid Size  \
Expiration Strike Type                                                         
2023-02-01 320.0  Call  0.0000   85.84   93.33   85.72       84.79       100   
                  Put   0.0000    0.01    0.01    0.01        0.00         0   
           330.0  Call  0.0000   74.77   82.97   74.77       74.80       100   
                  Put   0.0000    0.00    0.00    0.00        0.00         0   
           331.0  Call  0.0000   73.77   82.33   73.77       73.80       100   
...                        ...     ...     ...     ...         ...       ...   
2025-12-19 640.0  Put   0.0000    0.00    0.00    0.00      235.00       200   
           645.0  Call  0.9284    3.10    3.10    3.10        2.98       200   
                  Put   0.0000    0.00    0.00    0.00      240.00       200   
           650.0  Call  0.8856    3.40    3.88    3.40        3.05        11   
                  Put   0.0000  235.98  235.98  235.98      245.00       201   

                           Bid     Ask  Ask Size            Timestamp  
Expiration Strike Type                                                 
2023-02-01 320.0  Call   90.90   92.04       100  2023-02-01T15:42:28  
                  Put     0.00    0.01       893  2023-02-01T10:26:34  
           330.0  Call   80.90   82.04       100  2023-02-01T15:39:55  
                  Put     0.00    0.01       894  2023-01-26T09:49:23  
           331.0  Call   79.90   81.04       100  2023-02-01T15:42:28  
...                        ...     ...       ...                  ...  
2025-12-19 640.0  Put   223.66  233.66       200                 None  
           645.0  Call    0.01    9.60         1  2023-02-01T09:30:16  
                  Put   228.55  238.55       200                 None  
           650.0  Call    3.00    3.89         8  2023-02-01T15:57:29  
                  Put   233.16  243.15       200  2023-02-01T16:12:15  

[8334 rows x 27 columns]

In [24]:
calls,puts = separate_chains(chains)

In [25]:
calls

DTE       Tick  Last Price  % Change  Theoretical  \
Expiration Strike Type                                                       
2023-02-01 320.0  Call     0         up       93.33   10.0719        91.52   
           330.0  Call     0         up       82.97   10.9225        81.52   
           331.0  Call     0         up       82.33   11.5583        80.52   
           332.0  Call     0  No Change        0.00    0.0000        79.52   
           333.0  Call     0  No Change        0.00    0.0000        78.52   
...                      ...        ...         ...       ...          ...   
2025-12-19 630.0  Call  1052       down        4.80    0.0000         4.39   
           635.0  Call  1052         up        3.44    0.0000         4.14   
           640.0  Call  1052       down        4.00    0.0000         3.92   
           645.0  Call  1052         up        3.10    3.8526         3.71   
           650.0  Call  1052       down        3.60   18.0328         3.51   

                        $ to Spot  % to Spot  Breakeven  Vol   OI  ...  \
Expiration Strike Type                                             ...   
2023-02-01 320.0  Call       1.24     0.3019     412.04    6    0  ...   
           330.0  Call       1.24     0.3019     412.04    5    0  ...   
           331.0  Call       1.24     0.3019     412.04    4    0  ...   
           332.0  Call       1.24     0.3019     412.04    0    0  ...   
           333.0  Call       1.24     0.3019     412.04    0    0  ...   
...                           ...        ...        ...  ...  ...  ...   
2025-12-19 630.0  Call     229.20    55.7936     640.00    0    3  ...   
           635.0  Call     234.20    57.0107     645.00    0    5  ...   
           640.0  Call     239.00    58.1792     649.80    0    1  ...   
           645.0  Call     243.80    59.3476     654.60    1   15  ...   
           650.0  Call     243.09    59.1748     653.89   22  207  ...   

                           Rho   Open   High    Low  Prev Close  Bid Size  \
Expiration Strike Type                                                      
2023-02-01 320.0  Call  0.0000  85.84  93.33  85.72       84.79       100   
           330.0  Call  0.0000  74.77  82.97  74.77       74.80       100   
           331.0  Call  0.0000  73.77  82.33  73.77       73.80       100   
           332.0  Call  0.0000   0.00   0.00   0.00       72.80       100   
           333.0  Call  0.0000   0.00   0.00   0.00       71.80       100   
...                        ...    ...    ...    ...         ...       ...   
2025-12-19 630.0  Call  1.0750   0.00   0.00   0.00        3.80       200   
           635.0  Call  1.0235   0.00   0.00   0.00        3.56       200   
           640.0  Call  0.9745   0.00   0.00   0.00        3.38       200   
           645.0  Call  0.9284   3.10   3.10   3.10        2.98       200   
           650.0  Call  0.8856   3.40   3.88   3.40        3.05        11   

                          Bid    Ask  Ask Size            Timestamp  
Expiration Strike Type                                               
2023-02-01 320.0  Call  90.90  92.04       100  2023-02-01T15:42:28  
           330.0  Call  80.90  82.04       100  2023-02-01T15:39:55  
           331.0  Call  79.90  81.04       100  2023-02-01T15:42:28  
           332.0  Call  78.90  80.04       100                 None  
           333.0  Call  77.90  79.04       100                 None  
...                       ...    ...       ...                  ...  
2025-12-19 630.0  Call   1.59  10.00         8  2023-01-17T15:38:59  
           635.0  Call   1.03  10.00         8  2023-01-30T10:50:10  
           640.0  Call   1.23   9.80         8  2023-01-27T11:17:30  
           645.0  Call   0.01   9.60         1  2023-02-01T09:30:16  
           650.0  Call   3.00   3.89         8  2023-02-01T15:57:29  

[4167 rows x 27 columns]

In [26]:
puts

DTE       Tick  Last Price  % Change  Theoretical  \
Expiration Strike Type                                                       
2023-02-01 320.0  Put      0  No Change        0.01  100.0000         0.00   
           330.0  Put      0       down        0.02    0.0000         0.00   
           331.0  Put      0       down        0.03    0.0000         0.00   
           332.0  Put      0  No Change        0.02    0.0000         0.00   
           333.0  Put      0  No Change        0.01    0.0000         0.00   
...                      ...        ...         ...       ...          ...   
2025-12-19 630.0  Put   1052  No Change        0.00    0.0000       218.48   
           635.0  Put   1052  No Change        0.00    0.0000       223.48   
           640.0  Put   1052  No Change        0.00    0.0000       228.48   
           645.0  Put   1052  No Change        0.00    0.0000       233.48   
           650.0  Put   1052       down      235.98   -3.6816       238.48   

                        $ to Spot  % to Spot  Breakeven  Vol   OI  ...  Rho  \
Expiration Strike Type                                             ...        
2023-02-01 320.0  Put      -90.81   -22.1056     319.99    2  293  ...  0.0   
           330.0  Put      -80.81   -19.6714     329.99    0  355  ...  0.0   
           331.0  Put      -79.81   -19.4279     330.99    0  134  ...  0.0   
           332.0  Put      -78.81   -19.1845     331.99    0   14  ...  0.0   
           333.0  Put      -77.81   -18.9411     332.99    0  608  ...  0.0   
...                           ...        ...        ...  ...  ...  ...  ...   
2025-12-19 630.0  Put       -4.74    -1.1538     406.06    0    0  ...  0.0   
           635.0  Put       -4.44    -1.0808     406.36    0    0  ...  0.0   
           640.0  Put       -4.46    -1.0857     406.34    0    0  ...  0.0   
           645.0  Put       -4.35    -1.0589     406.45    0    0  ...  0.0   
           650.0  Put       -3.95    -0.9615     406.85    1    1  ...  0.0   

                          Open    High     Low  Prev Close  Bid Size     Bid  \
Expiration Strike Type                                                         
2023-02-01 320.0  Put     0.01    0.01    0.01         0.0         0    0.00   
           330.0  Put     0.00    0.00    0.00         0.0         0    0.00   
           331.0  Put     0.00    0.00    0.00         0.0         0    0.00   
           332.0  Put     0.00    0.00    0.00         0.0         0    0.00   
           333.0  Put     0.00    0.00    0.00         0.0         0    0.00   
...                        ...     ...     ...         ...       ...     ...   
2025-12-19 630.0  Put     0.00    0.00    0.00       225.0       200  213.94   
           635.0  Put     0.00    0.00    0.00       230.0       200  218.64   
           640.0  Put     0.00    0.00    0.00       235.0       200  223.66   
           645.0  Put     0.00    0.00    0.00       240.0       200  228.55   
           650.0  Put   235.98  235.98  235.98       245.0       201  233.16   

                           Ask  Ask Size            Timestamp  
Expiration Strike Type                                         
2023-02-01 320.0  Put     0.01       893  2023-02-01T10:26:34  
           330.0  Put     0.01       894  2023-01-26T09:49:23  
           331.0  Put     0.01       893  2023-01-25T10:33:22  
           332.0  Put     0.01       893  2023-01-25T15:50:34  
           333.0  Put     0.01       893  2023-01-30T10:34:48  
...                        ...       ...                  ...  
2025-12-19 630.0  Put   223.94       200                 None  
           635.0  Put   228.64       200                 None  
           640.0  Put   233.66       200                 None  
           645.0  Put   238.55       200                 None  
           650.0  Put   243.15       200  2023-02-01T16:12:15  

[4167 rows x 27 columns]

In [27]:
by_strike = calc_chains_by_strike(chains)

by_strike

,Call OI,Put OI,Net OI,Call Vol,Put Vol,Net Vol,Call Delta $,Put Delta $,Net Delta $,Call GEX,Put GEX,Net GEX
Strike,,,,,,,,,,,,
120.0,20,6761,6781,1,62,63,819956,-2055289,-1235333,337,-114096,-113759
130.0,19,303,322,0,0,0,777241,-110780,666461,320,-5113,-4793
140.0,0,196,196,0,0,0,0,-86958,-86958,0,-3307,-3307
150.0,398,125579,125977,7,569,576,16231420,-16496758,-265338,6107,-627670,-621563
155.0,41,9328,9369,0,211,211,1683322,-1424537,258785,83,-99835,-99752
...,...,...,...,...,...,...,...,...,...,...,...,...
700.0,19548,110,19658,75,509,584,4051849,-4518800,-466951,576893,0,576893
705.0,1653,0,1653,0,11,11,118576,0,118576,11744,0,11744
710.0,1148,0,1148,0,20,20,196632,0,196632,27049,0,27049


In [28]:
by_expiration = calc_chains_by_expiration(chains)

by_expiration

,Call OI,Put OI,Net OI,OI Ratio,Call Vol,Put Vol,Net Vol,Vol Ratio,Vol-OI Ratio,Call Delta $,Put Delta $,Net Delta $,Call GEX,Put GEX,Net GEX
Expiration,,,,,,,,,,,,,,,
2023-02-01,211952,282104,494056,1.3310,1803935,1428722,3232657,0.7920,6.5431,6469441023,-82946580,6386494443,1290618270,-218770950,1071847320
2023-02-02,172068,217831,389899,1.2660,973812,813742,1787554,0.8356,4.5847,4785573202,-535132865,4250440337,987534100,-590141489,397392611
2023-02-03,375284,806311,1181595,2.1485,540968,708643,1249611,1.3100,1.0576,9022835261,-1098795884,7924039377,1555133611,-837444109,717689502
2023-02-06,72768,88026,160794,1.2097,156627,132355,288982,0.8450,1.7972,1919636389,-462492504,1457143885,365801411,-265308243,100493168
2023-02-07,41436,55239,96675,1.3331,76527,71306,147833,0.9318,1.5292,1093457562,-442560960,650896602,197049750,-205060371,-8010621
2023-02-08,45182,55062,100244,1.2187,86506,85953,172459,0.9936,1.7204,1165715451,-443325798,722389653,199858302,-196050798,3807504
2023-02-09,29423,43433,72856,1.4762,35526,42714,78240,1.2023,1.0739,784623332,-327174546,457448786,131658743,-139071855,-7413112
2023-02-10,243653,584192,827845,2.3976,159613,224130,383743,1.4042,0.4635,5333194000,-1095755945,4237438055,1054460120,-494131682,560328438
2023-02-13,2622,5879,8501,2.2422,14525,17226,31751,1.1860,3.7350,58254395,-95893777,-37639382,10672024,-24717690,-14045666
